<a href="https://colab.research.google.com/github/OmarSyedK/AI-Resume-Optimization/blob/main/resumeopt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q google-generativeai

import google.generativeai as genai
from google.colab import userdata

#Fetch the API Key
api_key = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=api_key)

#In case the API key is not found
if not api_key:
  print("ERROR: GOOGLE_API_KEY not found")


In [5]:
#Prompt


#Markdown Resume
md_resume = input("Markdown Resume: ")


#Job Description
job_description = input("Job Description: ")

prompt = f"""
I am providing you a markdown resume and a job description \
I want you to optimize the resume according to the job role and job description \
Make relevent changes to the resume in sections such as skills, projects, achievments, etc \
while keeping the unique qualifications and strengths. \
The output should state the changes you made to the resume and why you made them. \
Then return the optimized resume in markdown format.

### Here is the resume in Markdown:
{md_resume}

### Here is the job description:
{job_description}

Please modify the resume to:
- Contain keywords and phrases relevent to the job description
- Make sure the experiences are presented in a way that match the job description requirements.
- Maintain clarity, conciseness, and professionalism throughout.

"""



Markdown Resume: # Omar Syed Kaiser Hyderabad, Telangana   📞 7416059130 | 📧 omarsyedk419@gmail.com    ---  ## About Me - Engaging in practical projects and internships to apply theoretical knowledge in real-world scenarios.   - Learning to work effectively in teams and improving communication skills.   - Staying updated with emerging technologies and industry trends to adapt to the evolving landscape.    ---  ## Education **B.Tech in Computer Science Engineering (AI & ML)**   *Lord's Institute of Engineering and Technology, Hyderabad, Telangana*   2023 – 2027   - GPA: **8/10** (After 3 semesters)   - Relevant coursework: Java Programming, Python Programming, C Programming, Soft Skills & Employability Skills    ---  ## Skills - **Programming Languages:** Python, Java, C   - **Mobile App Development:** Flutter, Dart, Firebase   - **Web Technologies:** HTML, CSS, JavaScript (basic)   - **Tools & Platforms:** Git, GitHub, OpenCV, Google Colab, VS Code, Dialogflow   - **Concepts:** Data Str

In [6]:
# ---Model and generation configuration---
system_instruction = "You are a helpful assistant and an expert in career coaching specializing in tailoring resumes."

#Setting up the model
model = genai.GenerativeModel(
  model_name = 'gemini-2.5-flash-lite',
  system_instruction = system_instruction
)

# Configuration for the generation call
generation_config = genai.GenerationConfig(
    temperature=0.25
)

In [7]:
# ---Make API Call---
try:
  print(f"Optimizing your resume for {job_description}...")
  response = model.generate_content(
  prompt,
  generation_config = generation_config
  )
# --- Extract Response---
  print(response.text)
except Exception as e:
  print(f"Error: {e}")

Optimizing your resume for Backend Developer...
Okay, I can help you optimize your resume for a Backend Developer role.

Here's a breakdown of the changes I've made and why:

**Changes Made and Rationale:**

1.  **"About Me" Section:**
    *   **Change:** Rephrased to highlight backend-relevant skills and aspirations. Removed general statements about learning and teamwork to focus on technical contributions.
    *   **Why:** The original "About Me" was too generic. For a backend role, it's crucial to immediately signal your interest and foundational understanding of backend development principles, even if your experience is primarily project-based.

2.  **"Skills" Section:**
    *   **Change:**
        *   Added "Backend Development" as a primary skill category.
        *   Reorganized and emphasized relevant technologies.
        *   Moved "Firebase" to a more prominent position as it's often used in backend contexts (e.g., BaaS).
        *   Added "API Development" and "Database Mana